In [1]:
%autoawait
import time
import asyncio
import base64

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [3]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


Results :  [{'initiator': 'self', 'routing_state': 'none', 'invitation_key': '5a6zsXxQkrnrFpZhnZ6phfZMqWmbM9GMTntwcfmUQNAM', 'updated_at': '2020-09-10 07:03:30.846826Z', 'my_did': 'DKucuEdKa4HC156LpgqLNo', 'created_at': '2020-09-10 07:03:14.919425Z', 'their_did': 'QzoMbdwg9Ppt8BBr3LT81y', 'their_label': 'Alice', 'accept': 'manual', 'state': 'active', 'connection_id': '049d9971-582d-4b8c-9ef5-12b1dc7ae2d6', 'invitation_mode': 'once'}]
Connection : {'initiator': 'self', 'routing_state': 'none', 'invitation_key': '5a6zsXxQkrnrFpZhnZ6phfZMqWmbM9GMTntwcfmUQNAM', 'updated_at': '2020-09-10 07:03:30.846826Z', 'my_did': 'DKucuEdKa4HC156LpgqLNo', 'created_at': '2020-09-10 07:03:14.919425Z', 'their_did': 'QzoMbdwg9Ppt8BBr3LT81y', 'their_label': 'Alice', 'accept': 'manual', 'state': 'active', 'connection_id': '049d9971-582d-4b8c-9ef5-12b1dc7ae2d6', 'invitation_mode': 'once'}
Active Connection ID :  049d9971-582d-4b8c-9ef5-12b1dc7ae2d6


In [4]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    base64_bytes = data.encode('ascii')
    text = base64.b64decode(base64_bytes)
    message = text.decode('ascii')
    f = open("test.txt","w+")
    f.write(message)
    f.close()
    
#     data = payload['content'] 
#     asyncio.get_event_loop().create_task(om_controller.messaging.send_message(connection_id, " Received!"))
message_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([message_listener], defaults=True)


Handle Attachment {'connection_id': '049d9971-582d-4b8c-9ef5-12b1dc7ae2d6', 'message_id': 'abc64ece-8677-4117-a3c5-8addc244a5ba', 'attachment_message': 'Here is the data you wanted', 'content': 'VGhpcyBpcyBhIHRlc3QgZmlsZSB0byBhdHRhY2gu', 'filename': 'test_file.txt', 'state': 'received'} 049d9971-582d-4b8c-9ef5-12b1dc7ae2d6
Attachment Received


In [6]:
response = await om_controller.protocol.send_attachment(connection_id,data)

print(response)

TypeError: send_attachment() missing 1 required positional argument: 'data'

In [7]:
response = await om_controller.terminate()
print(response)


None
